## Library Load

In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy as cp
import Agile_data
import os , re
import warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf
import copy as cp
#tf.enable_eager_execution()

In [5]:
_, test_categorical, test_continuous, test_Segment, \
test_label , test_label_eval  = Agile_data.load_test_data()
_, Deep_data_test, test_label = Agile_data.load_Wide_Deep_test_data()

## 변형한 데이터는 ㄷㄷ 똑같음

In [46]:
Input = pd.read_csv("Total_Data_ver2.csv" , nrows = 1)
test_ = pd.read_csv("Test_Data_ver2.csv")

In [47]:
columns = Input.columns.tolist()
Continuous_col1 = [ i for i in columns if re.search("D2V" , i) is not None ]
Continuous_col2 = [ i for i in columns if re.search("[A-D][0-9]{2}" , i) is not None ]
Continuous_col3 = ["dadungi" , "Deoyoung" ,"Robs", "himart"]
Categorical_Seg = [ i for i in columns if re.search("^X" , i) is not None ]
Categorical_col = ["Gender" , "AgeGroup" ,"A_residential_area"] 

In [48]:
assert test_.columns.tolist() + ["target"] == Input.columns.tolist()

In [49]:
columns = Continuous_col1 + Categorical_col + Continuous_col3 + Continuous_col2 + Categorical_Seg
assert len(Input.columns) == len(columns) + 1

In [51]:
Continuous_col = cp.deepcopy(Continuous_col1)
Continuous_col.extend(Continuous_col2) 
Continuous_col.extend(Continuous_col3)

## Continuous Column

---

In [20]:
print(Continuous_col)

['D2V_0', 'D2V_1', 'D2V_2', 'D2V_3', 'D2V_4', 'D2V_5', 'D2V_6', 'D2V_7', 'D2V_8', 'D2V_9', 'D2V_10', 'D2V_11', 'D2V_12', 'D2V_13', 'D2V_14', 'D2V_15', 'D2V_16', 'D2V_17', 'D2V_18', 'D2V_19', 'D2V_20', 'D2V_21', 'D2V_22', 'D2V_23', 'D2V_24', 'D2V_25', 'D2V_26', 'D2V_27', 'D2V_28', 'D2V_29', 'D2V_30', 'D2V_31', 'D2V_32', 'D2V_33', 'D2V_34', 'D2V_35', 'D2V_36', 'D2V_37', 'D2V_38', 'D2V_39', 'D2V_40', 'D2V_41', 'D2V_42', 'D2V_43', 'D2V_44', 'D2V_45', 'D2V_46', 'D2V_47', 'D2V_48', 'D2V_49', 'D2V_50', 'D2V_51', 'D2V_52', 'D2V_53', 'D2V_54', 'D2V_55', 'D2V_56', 'D2V_57', 'D2V_58', 'D2V_59', 'D2V_60', 'D2V_61', 'D2V_62', 'D2V_63', 'D2V_64', 'D2V_65', 'D2V_66', 'D2V_67', 'D2V_68', 'D2V_69', 'D2V_70', 'D2V_71', 'D2V_72', 'D2V_73', 'D2V_74', 'D2V_75', 'D2V_76', 'D2V_77', 'D2V_78', 'D2V_79', 'D2V_80', 'D2V_81', 'D2V_82', 'D2V_83', 'D2V_84', 'D2V_85', 'D2V_86', 'D2V_87', 'D2V_88', 'D2V_89', 'D2V_90', 'D2V_91', 'D2V_92', 'D2V_93', 'D2V_94', 'D2V_95', 'D2V_96', 'D2V_97', 'D2V_98', 'D2V_99', 'D2V_100'

In [21]:
Continous_base =\
[ tf.contrib.layers.real_valued_column( continuous ) for continuous in Continuous_col ]


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



## Segment Column

---

In [23]:
print(Categorical_Seg)

['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17']


In [24]:
Categorical_base = []
for i in Categorical_Seg :
    key = Input[i].astype(str).unique().tolist()
    if len(key) < 5 :
        value = tf.contrib.layers.sparse_column_with_keys(i , keys = key)
        Categorical_base.append(value)
    else :
        bucket_size = 1000 if len(key) > 10 else 100
#         value = tf.feature_column.categorical_column_with_vocabulary_list(
#             key= i ,vocabulary_list= key )
        value = tf.contrib.layers.sparse_column_with_hash_bucket(i , hash_bucket_size= bucket_size  )
        Categorical_base.append(value)

## Cross Column

---

In [25]:
Seg = Categorical_Seg
Seg_dict = dict(zip(Seg , list(np.arange(len(Seg)))))

In [27]:
cross_col = [[Seg[i], Seg[i + j + 1]]
        for i in range(16) for j in range(16-i)]
Cross_Columns = []
for i , j in cross_col :
    id1 , id2 = Seg_dict[i] , Seg_dict[j]
    cross_tf = tf.contrib.layers.crossed_column\
    ([Categorical_base[id1], Categorical_base[id2]],
     hash_bucket_size=int(1e4))
    Cross_Columns.append(cross_tf)

In [28]:
print(cross_col)

[['X1', 'X2'], ['X1', 'X3'], ['X1', 'X4'], ['X1', 'X5'], ['X1', 'X6'], ['X1', 'X7'], ['X1', 'X8'], ['X1', 'X9'], ['X1', 'X10'], ['X1', 'X11'], ['X1', 'X12'], ['X1', 'X13'], ['X1', 'X14'], ['X1', 'X15'], ['X1', 'X16'], ['X1', 'X17'], ['X2', 'X3'], ['X2', 'X4'], ['X2', 'X5'], ['X2', 'X6'], ['X2', 'X7'], ['X2', 'X8'], ['X2', 'X9'], ['X2', 'X10'], ['X2', 'X11'], ['X2', 'X12'], ['X2', 'X13'], ['X2', 'X14'], ['X2', 'X15'], ['X2', 'X16'], ['X2', 'X17'], ['X3', 'X4'], ['X3', 'X5'], ['X3', 'X6'], ['X3', 'X7'], ['X3', 'X8'], ['X3', 'X9'], ['X3', 'X10'], ['X3', 'X11'], ['X3', 'X12'], ['X3', 'X13'], ['X3', 'X14'], ['X3', 'X15'], ['X3', 'X16'], ['X3', 'X17'], ['X4', 'X5'], ['X4', 'X6'], ['X4', 'X7'], ['X4', 'X8'], ['X4', 'X9'], ['X4', 'X10'], ['X4', 'X11'], ['X4', 'X12'], ['X4', 'X13'], ['X4', 'X14'], ['X4', 'X15'], ['X4', 'X16'], ['X4', 'X17'], ['X5', 'X6'], ['X5', 'X7'], ['X5', 'X8'], ['X5', 'X9'], ['X5', 'X10'], ['X5', 'X11'], ['X5', 'X12'], ['X5', 'X13'], ['X5', 'X14'], ['X5', 'X15'], ['X5', 'X

Categorical Column
---

---

In [ ]:
from IPython.display import Image
Image("./images/Categorical_Embedding.PNG")

In [31]:
print(Categorical_col)

['Gender', 'AgeGroup', 'A_residential_area']


In [32]:
Categorical_base2 =\
[ tf.contrib.layers.sparse_column_with_keys( Cat , keys = Input[Cat].unique().tolist() ) for Cat in Categorical_col ]

Wide Columns
---

Categorical Column + Segment Column + Cross Column + Doc2Vec

--- 

In [283]:
wide_columns  = Categorical_base + Categorical_base2 + Cross_Columns + Continous_base[0:300]

#https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/tutorials/wide_and_deep/

## Embedding Column

---

In [34]:
embed_col = []
for idx , i in enumerate(Categorical_col) :
    key = Categorical_base2[idx]
    k = 8
    dim = int( np.ceil( np.sqrt(Input[i].nunique()) ) ) * k
    emb = tf.contrib.layers.embedding_column(key , dimension=dim)
    embed_col.append(emb)

In [286]:
embed_col

[_EmbeddingColumn(sparse_id_column=_SparseColumnKeys(column_name='Gender', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('F',), num_oov_buckets=0, vocab_size=1, default_value=-1), combiner='sum', dtype=tf.string), dimension=8, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7f442ee760f0>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, shared_embedding_name=None, shared_vocab_size=None, max_norm=None, trainable=True),
 _EmbeddingColumn(sparse_id_column=_SparseColumnKeys(column_name='AgeGroup', is_integerized=False, bucket_size=None, lookup_config=_SparseIdLookupConfig(vocabulary_file=None, keys=('50세~54세',), num_oov_buckets=0, vocab_size=1, default_value=-1), combiner='sum', dtype=tf.string), dimension=8, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7f442ee76c18>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, shared_embedding_name=None, sh

Deep Columns
---

Embedding Column + Continuous Column

--- 

In [66]:
deep_columns = embed_col + Continous_base

# Train Model

In [294]:
inter_op = 8
intra_op = 8
config = tf.ConfigProto(device_count={'GPU': 0},
                        log_device_placement=True , 
                        inter_op_parallelism_threads=inter_op,
                        intra_op_parallelism_threads=intra_op)
config.gpu_options.allow_growth = True
run_config = tf.estimator.RunConfig().replace(
    session_config=config)
model_dir = "/var/disk/WideDeep_ver2"
m = tf.estimator.DNNLinearCombinedClassifier(
    model_dir=model_dir,
    linear_feature_columns=wide_columns,
    dnn_feature_columns=deep_columns,
    dnn_activation_fn=tf.nn.leaky_relu ,
    n_classes = test_label_eval.shape[1] , 
    batch_norm = True,
    dnn_dropout = 0.3,
    dnn_hidden_units=[200 ,  150 , 100 , 50],
    dnn_optimizer=tf.train.AdamOptimizer(learning_rate = 0.01) , 
    linear_optimizer=tf.train.AdamOptimizer(learning_rate = 0.01),
    config=run_config
) 
#     

INFO:tensorflow:Using config: {'_model_dir': '/var/disk/WideDeep_ver2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': device_count {
  key: "GPU"
}
intra_op_parallelism_threads: 8
inter_op_parallelism_threads: 8
gpu_options {
  allow_growth: true
}
log_device_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f4036b21048>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


# tf.decode_csv 버전 
## Error 발생 해결1[URL](https://stackoverflow.com/questions/39730528/typeerror-signature-mismatch-keys-must-be-dtype-dtype-string-got-dtype)

In [288]:
_CSV_COLUMNS = Input.columns.tolist()
_CSV_COLUMN_DEFAULTS = [[0.0]] * len(Continuous_col1) +\
[[""]] * len(Categorical_col) +\
[[0.0]] * len(Continuous_col3 + Continuous_col2) +\
[[""]] * len(Categorical_Seg) +\
[[0]] * 1

In [289]:
assert len(_CSV_COLUMNS) == len(_CSV_COLUMN_DEFAULTS)

In [290]:
Columns = Input.columns.tolist()
CONTINUOUS_COLUMNS = Continuous_col
CATEGORICAL_COLUMNS = Categorical_col +  Categorical_Seg

In [291]:
print("Categorical : \n" ,  CATEGORICAL_COLUMNS)
print(" ")
print("Continuous : \n" ,  CONTINUOUS_COLUMNS)

Categorical : 
 ['Gender', 'AgeGroup', 'A_residential_area', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17']
 
Continuous : 
 ['D2V_0', 'D2V_1', 'D2V_2', 'D2V_3', 'D2V_4', 'D2V_5', 'D2V_6', 'D2V_7', 'D2V_8', 'D2V_9', 'D2V_10', 'D2V_11', 'D2V_12', 'D2V_13', 'D2V_14', 'D2V_15', 'D2V_16', 'D2V_17', 'D2V_18', 'D2V_19', 'D2V_20', 'D2V_21', 'D2V_22', 'D2V_23', 'D2V_24', 'D2V_25', 'D2V_26', 'D2V_27', 'D2V_28', 'D2V_29', 'D2V_30', 'D2V_31', 'D2V_32', 'D2V_33', 'D2V_34', 'D2V_35', 'D2V_36', 'D2V_37', 'D2V_38', 'D2V_39', 'D2V_40', 'D2V_41', 'D2V_42', 'D2V_43', 'D2V_44', 'D2V_45', 'D2V_46', 'D2V_47', 'D2V_48', 'D2V_49', 'D2V_50', 'D2V_51', 'D2V_52', 'D2V_53', 'D2V_54', 'D2V_55', 'D2V_56', 'D2V_57', 'D2V_58', 'D2V_59', 'D2V_60', 'D2V_61', 'D2V_62', 'D2V_63', 'D2V_64', 'D2V_65', 'D2V_66', 'D2V_67', 'D2V_68', 'D2V_69', 'D2V_70', 'D2V_71', 'D2V_72', 'D2V_73', 'D2V_74', 'D2V_75', 'D2V_76', 'D2V_77', 'D2V_78', 'D2V_79', 'D2V_80', 'D2V_81', '

In [292]:
def input_fn(data_file, num_epochs, shuffle, batch_size , Eval ):
    def parse_csv(data_file ):
        print('Parsing', data_file)
        print(Eval)
        if Eval == "train" :
            columns = tf.decode_csv(data_file , record_defaults=_CSV_COLUMN_DEFAULTS)
            features = dict(zip(_CSV_COLUMNS, columns))
            labels = features.pop('target')
        elif Eval == "test" :
            columns = tf.decode_csv(data_file , record_defaults=_CSV_COLUMN_DEFAULTS[:-1])
            features = dict(zip(_CSV_COLUMNS[:-1] , columns))
            labels = tf.constant([1] , dtype = tf.int64 , shape = () ) # , name ="target" 
        return features, labels
    
    dataset = tf.data.TextLineDataset(data_file).skip(1)
    if shuffle :
        dataset = dataset.shuffle(buffer_size=10000) 
    dataset = dataset.map(parse_csv, num_parallel_calls=10)
    if Eval == "train" :
        dataset = dataset.repeat(num_epochs)
        dataset = dataset.batch(batch_size , drop_remainder= True)
    else :
        dataset = dataset.batch(batch_size , drop_remainder= True)
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

def train_input_fn():
    return input_fn(data_file = "./Total_Data_ver2.csv" ,
                    num_epochs= 10 ,
                    shuffle= True ,
                    batch_size= 256,
                    Eval = "train"
                   )

def csv_input(data):
    continuous_cols = {k: tf.constant(data[k].values)
                       for k in CONTINUOUS_COLUMNS}
    categorical_cols = {k: tf.SparseTensor( indices=[[i, 0] for i in range(data[k].size)], values=data[k].astype(str).values , 
                                           dense_shape=[data[k].size, 1] ) for k in CATEGORICAL_COLUMNS}

    feature_cols = {**continuous_cols , **categorical_cols}
    labels = None
    return feature_cols , labels

def csv_input_fn():
    csv = csv_input(data = test_)    
    return csv

## Train

In [ ]:
m.train(input_fn=train_input_fn , 
        max_steps= 10000 ,)

Parsing Tensor("arg0:0", shape=(), dtype=string, device=/device:CPU:0)
train
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:loss = 1748.514, step = 1
INFO:tensorflow:Saving checkpoints for 75 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.124863
INFO:tensorflow:loss = 1284.9064, step = 101 (800.881 sec)
INFO:tensorflow:Saving checkpoints for 155 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.136656
INFO:tensorflow:loss = 1308.0979, step = 201 (731.760 sec)
INFO:tensorflow:Saving checkpoints for 236 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.123011
INFO:tensorflow:loss = 1358.6477, step = 301 (812.935 sec)
INFO:ten

INFO:tensorflow:global_step/sec: 0.122883
INFO:tensorflow:loss = 1138.9651, step = 3401 (813.784 sec)
INFO:tensorflow:Saving checkpoints for 3434 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.126288
INFO:tensorflow:loss = 1126.6016, step = 3501 (791.841 sec)
INFO:tensorflow:Saving checkpoints for 3506 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:Saving checkpoints for 3579 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.110636
INFO:tensorflow:loss = 1135.1769, step = 3601 (903.864 sec)
INFO:tensorflow:Saving checkpoints for 3651 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.129213
INFO:tensorflow:loss = 1176.3572, step = 3701 (773.918 sec)
INFO:tensorflow:Saving checkpoints for 3726 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:Saving checkpoints for 3799 into /var/disk/WideDeep_ver2/model.ckpt.
INFO:tensorflow:global_step/sec: 0.111138
INFO:tensorflow:loss = 1102.6376, step =

In [72]:
results = m.evaluate(input_fn=train_input_fn , steps=1)
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

Instructions for updating:
Colocations handled automatically by placer.
Parsing Tensor("arg0:0", shape=(), dtype=string, device=/device:CPU:0)
train
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
The default behavior of sparse_feature_cross is changing, the default
value for hash_key will change to SPARSE_FEATURE_CROSS_DEFAULT_HASH_KEY.
From that point on sparse_feature_cross will always use FingerprintCat64
to concatenate the feature fingerprints. And the underlying
_sparse_feature_cross_op.sparse_feature_cross operation will be marked
as deprecated.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-07-21T01:49:23Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /var/disk/WideDeep/model.ckpt-5411
INFO:tensorflow:Running local_in

## TEST CSV Version

## [Predict 해결법](https://stackoverflow.com/questions/46857382/tensorflow-estimator-predict-gives-warningtensorflowinput-graph-does-not-con)

In [116]:
import time
start = time.time()
csvProb = m.predict(input_fn= csv_input_fn , predict_keys="probabilities" )
for i in np.arange( test_.shape[0] ) :   
    prob_next = csvProb.__next__()["probabilities"].reshape(1, -1 )
    if i == 0 :
        print('Start')
        prob = prob_next
    else :
        prob = np.concatenate((prob,prob_next),axis = 0)
print("eval time : " , time.time() - start)    

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/disk/WideDeep/model.ckpt-5480
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Start
eval time :  213.02658796310425


In [117]:
prob.shape

(13915, 884)

In [107]:
def get_acc(score_matrix, top_n, test_matix):
    avg_acc = 0
    for i in range(len(score_matrix)):
        top = score_matrix.iloc[i].nlargest(top_n).index
        tmp = 0
        for j in range(len(top)):
            true_buy = [i.split("/")[0] for i in test_matix["item"][0].split()]
            if top[j] in true_buy :
                tmp += 1

        acc = tmp / min(len(top) , len(true_buy))
        avg_acc += acc / len(score_matrix)

    return print("Hit rate of Top {}: {:>.5f}".format(top_n, avg_acc))

## 몇개 이상 구매한 사람 기준으로 추천해줬을 때

---

### $$\frac{추천한\ 것\ 중에서\ 구매를\ 한\ 수 }{min(추천수 , 구매수) \quad \quad }$$


In [112]:
score_pd = pd.DataFrame(prob, columns=test_label_eval.columns)
cond = list(np.sum(test_label_eval.values , axis = 1) > 10 )
cond_pd = score_pd[cond].reset_index(drop= True)
test_pd = test_label[cond].reset_index(drop= True)

In [113]:
get_acc(cond_pd, 10 , test_pd)

Hit rate of Top 10: 0.12169


In [265]:
def get_acc_idx(prob, N , test_label_eval , idx , dict_store ) :
    def top_n(data , n ) :
        return list(map(lambda x: np.argpartition(x, -n)[-n:][0], data ))[0]
    true_label =\
    np.where(test_label_eval.iloc[idx : (idx+1),:].values == 1 )[1].tolist()    
    recommend = [top_n(prob , i) for i in np.arange(1,N+1)]
    true_label = np.where(test_label_eval.iloc[idx : (idx+1),:].values == 1 )[1].tolist()
    target = test_label_eval.columns.tolist()
    REC = [target[i] for i in recommend]
    REC = [dict_store[i] for i in  REC ]
    correct = 0
    for rec in recommend :
        if rec in true_label :
            correct += 1 
    acc = correct / len(recommend) 
    print("Hit rate of Top {}: {:>.5f}".format(N , acc))
    return acc , REC

## 15개를 구매한 유저 중에서 한 유저에 대한 분석

> 10개를 추천해줬을 때 얼마나 실제 구매한 것이 포함되어 있는지

In [266]:
list_ = []
for i in test_label["item"].values.tolist() :
    store = i.split()
    for j in store : 
        list_.append(j)
Store = np.unique(list_)
dict_store = {}
for st in Store :
    dict_store[st.split("/")[0]] = st

In [267]:
cond = list(np.sum(test_label_eval.values , axis = 1) == 3  )
user_pd = test_label[cond].reset_index(drop= True)
cond_pd = prob[cond]
test_eval_pd = test_label_eval[cond].reset_index(drop= True)
idx = 200
ACC , REC = get_acc_idx(cond_pd , 9 , test_eval_pd , idx , dict_store )

print("\n 추천 한 물품 : \n" , " | ".join(REC) )

Hit rate of Top 9: 0.11111

 추천 한 물품 : 
 C150302/쿠키 | C010110/딸기 | C170701/생활잡화균일가 | C150506/젤리 | C150405/일반스낵 | C010206/바나나 | C150306/파이 | C150103/국물용기라면 | C170206/종량제봉투


### ID : 10951

---

* 구매했던 상품들

상품 명             | 상품 명          | 상품 명
:-------------------|:-----------------|:------------------
C130204/스틱원두커피 |C170414/여성타이즈 | C170701/생활잡화균일가

<br>

* 추천 상품 

상품 명             | 상품 명          | 상품 명
:-------------------|:-----------------|:------------------
 C150302/쿠키 | C010110/딸기 | C170701/생활잡화균일가 
 C150506/젤리 | C150405/일반스낵 | C010206/바나나 
 C150306/파이 | C150103/국물용기라면 | C170206/종량제봉투
 
 <br>
 
* **Hit rate of Top 9: 0.11111**

### $\frac{추천해준\ 상품에서\ 실제\ 구매한\ 상품의\ 개수}{min(추천해준\ 상품의\ 개수 , 구매수) \quad \quad \quad }$



In [278]:
cond = list(np.sum(test_label_eval.values , axis = 1) == 1  )
user_pd = test_label[cond].reset_index(drop= True)
cond_pd = prob[cond]
test_eval_pd = test_label_eval[cond].reset_index(drop= True)
idx = 25
ACC , REC = get_acc_idx(cond_pd , 9 , test_eval_pd , idx , dict_store )

print("\n 추천 한 물품 : \n" , " | ".join(REC) )
print(" ")
print(user_pd.iloc[idx])

Hit rate of Top 9: 0.11111

 추천 한 물품 : 
 C060406/베이커리일반빵 | C060108/즉석반찬원부재료 | C060206/온장조리원부재료 | C060504/기타냉장조리 | C060402/일반빵 | C060205/즉석어묵 | C060506/일반떡 | C060403/호빵찐빵 | C060501/한과
 
item     C060402/일반빵
영수증번호         173825
고객번호           10198
Name: 25, dtype: object


### ID : 10198

---

* 구매했던 상품들

상품 명             | 
:-------------------|
C060402/일반빵 |

<br>

* 추천 상품 

상품 명             | 상품 명          | 상품 명
:-------------------|:-----------------|:------------------
 C060406/베이커리일반빵 | C060108/즉석반찬원부재료 | C060206/온장조리원부재료 
 C060504/기타냉장조리 | C060402/일반빵 | C060205/즉석어묵 
 C060506/일반떡 | C060403/호빵찐빵 | C060501/한과
 
 <br>
 
* **Hit rate of Top 9: 0.11111**

### $\frac{추천해준\ 상품에서\ 실제\ 구매한\ 상품의\ 개수}{min(추천해준\ 상품의\ 개수 , 구매수) \quad \quad \quad }$

> 10198번 고객은 __'일반빵'__을 1번 구매한 고객입니다.
이런 경우에 알고리즘에서는 __'베이커리일반빵' , '일반빵','일반떡'__등 상위카테고리 06 범주에서 비슷한 상품들을 추천해주는 모습입니다.